<picture>
  <source media="(prefers-color-scheme: dark)" srcset="https://vespa.ai/assets/vespa-ai-logo-heather.svg">
  <source media="(prefers-color-scheme: light)" srcset="https://vespa.ai/assets/vespa-ai-logo-rock.svg">
  <img alt="#Vespa" width="200" src="https://vespa.ai/assets/vespa-ai-logo-rock.svg" style="margin-bottom: 25px;">
</picture>

# Feeding performance

This explorative notebook intends to shine some light on the different modes of feeding documents to Vespa.
We will look at these 3 different methods:

1. Using `feed_iterable()`.
2. Using `feed_iterable_async()`
3. Using [Vespa CLI](https://docs.vespa.ai/en/vespa-cli)


<div class="alert alert-info">
    Refer to <a href="https://pyvespa.readthedocs.io/en/latest/troubleshooting.html">troubleshooting</a>
    for any problem when running this guide.
</div>


[Install pyvespa](https://pyvespa.readthedocs.io/), Vespa CLI, and start Docker Daemon.

> The `vespacli` python package is just a thin wrapper, allowing for installation through pypi.
> Do NOT install if you already have the Vespa CLI installed.


In [1]:
#!pip3 install pyvespa vespacli
#!docker info

## Create an application package

The [application package](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.package.ApplicationPackage)
has all the Vespa configuration files -
create one from scratch:


In [2]:
from vespa.package import (
    ApplicationPackage,
    Field,
    Schema,
    Document,
    FieldSet,
)

package = ApplicationPackage(
    name="pyvespafeed",
    schema=[
        Schema(
            name="doc",
            document=Document(
                fields=[
                    Field(name="id", type="string", indexing=["summary"]),
                    Field(name="text", type="string", indexing=["summary"]),
                ]
            ),
            fieldsets=[FieldSet(name="default", fields=["text"])],
        )
    ],
)

Note that the `ApplicationPackage` name cannot have `-` or `_`.


## Deploy the Vespa application

Deploy `package` on the local machine using Docker,
without leaving the notebook, by creating an instance of
[VespaDocker](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.deployment.VespaDocker). `VespaDocker` connects
to the local Docker daemon socket and starts the [Vespa docker image](https://hub.docker.com/r/vespaengine/vespa/).

If this step fails, please check
that the Docker daemon is running, and that the Docker daemon socket can be used by clients (Configurable under advanced settings in Docker Desktop).


In [3]:
from vespa.deployment import VespaDocker

vespa_docker = VespaDocker()
app = vespa_docker.deploy(application_package=package)

/Users/thomas/Repos/pyvespa/pyvespa/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Waiting for configuration server, 0/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 0/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 5/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 10/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Application is up!
Finished deployment.


`app` now holds a reference to a [Vespa](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.application.Vespa) instance.


## Preparing the data

In this example we use [HF Datasets](https://huggingface.co/docs/datasets/index) library to stream the
["Cohere/wikipedia-2023-11-embed-multilingual-v3"](https://huggingface.co/datasets/Cohere/wikipedia-2023-11-embed-multilingual-v3) dataset and index in our newly deployed Vespa instance.

The dataset contains wikipedia-pages, and their corresponding embeddings.

> For this exploration we will just use the `id` and `text`-fields

The following uses the [stream](https://huggingface.co/docs/datasets/stream) option of datasets to stream the data without
downloading all the contents locally.

The `map` functionality allows us to convert the
dataset fields into the expected feed format for `pyvespa` which expects a dict with the keys `id` and `fields`:

`{ "id": "vespa-document-id", "fields": {"vespa_field": "vespa-field-value"}}`


In [4]:
from datasets import load_dataset

dataset = load_dataset(
    "Cohere/wikipedia-2023-11-embed-multilingual-v3",
    "simple",
    split="train",
    streaming=True,
)

## Utility function to stream different number of documents


In [5]:
def get_dataset(n_docs: int = 1000):
    return (
        dataset.map(lambda x: {"id": x["_id"] + "-iter", "fields": {"text": x["text"]}})
        .select_columns(["id", "fields"])
        .take(n_docs)
    )

### A dataclass to store the parameters and results of the different feeding methods


In [6]:
from dataclasses import dataclass
from typing import Callable, Optional, Iterable, Dict


@dataclass
class FeedParams:
    name: str
    num_docs: int
    max_connections: int
    function_name: str
    max_workers: Optional[int] = None
    max_queue_size: Optional[int] = None


@dataclass
class FeedResult(FeedParams):
    feed_time: Optional[float] = None

### A common callback function to notify if something goes wrong


In [7]:
from vespa.io import VespaResponse


def callback(response: VespaResponse, id: str):
    if not response.is_successful():
        print(
            f"Failed to feed document {id} with status code {response.status_code}: Reason {response.get_json()}"
        )

### Defining our feeding functions


In [8]:
import time
import asyncio


def feed_sync(params: FeedParams, data: Iterable[Dict]) -> FeedResult:
    start_time = time.time()
    with app.syncio(connections=params.max_connections):
        for doc in data:
            app.feed_data_point(
                data_id=doc["id"],
                fields=doc["fields"],
                schema="doc",
                namespace="pyvespa-feed",
                callback=callback,
            )
    end_time = time.time()
    return FeedResult(
        **params.__dict__,
        feed_time=end_time - start_time,
    )


async def feed_async(params: FeedParams, data: Iterable[Dict]) -> FeedResult:
    start_time = time.time()
    tasks = []
    async with app.asyncio(
        connections=params.max_connections, total_timeout=params.num_docs // 10
    ) as async_app:
        for doc in data:
            task = asyncio.create_task(
                async_app.feed_data_point(
                    data_id=doc["id"],
                    fields=doc["fields"],
                    schema="doc",
                    namespace="pyvespa-feed",
                    timeout=10,
                )
            )
            tasks.append(task)
        await asyncio.wait(tasks, return_when=asyncio.ALL_COMPLETED)
    end_time = time.time()
    return FeedResult(
        **params.__dict__,
        feed_time=end_time - start_time,
    )


def feed_iterable(params: FeedParams, data: Iterable[Dict]) -> FeedResult:
    start = time.time()
    app.feed_iterable(
        data,
        schema="doc",
        namespace="pyvespa-feed",
        operation_type="feed",
        max_queue_size=params.max_queue_size,
        max_workers=params.max_workers,
        max_connections=params.max_connections,
        callback=callback,
    )
    end = time.time()
    sync_feed_time = end - start
    return FeedResult(
        **params.__dict__,
        feed_time=sync_feed_time,
    )

## Defining our hyperparameters


In [9]:
from itertools import product

num_docs = [1000, 10_000, 100_000]

params_by_function = {
    "feed_sync": {
        "num_docs": num_docs,
        "max_connections": [16, 64],
    },
    "feed_async": {
        "num_docs": num_docs,
        "max_connections": [16, 64],
    },
    "feed_iterable": {
        "num_docs": num_docs,
        "max_connections": [64, 128],
        "max_workers": [16, 64],
        "max_queue_size": [1000, 10000],
    },
}

feed_params = []
# Create one FeedParams instance of each permutation
for func, parameters in params_by_function.items():
    print(f"Function: {func}")
    keys, values = zip(*parameters.items())
    for combination in product(*values):
        settings = dict(zip(keys, combination))
        print(settings)
        feed_params.append(
            FeedParams(
                name=f"{settings['num_docs']}_{settings['max_connections']}_{settings.get('max_workers', 0)}_{func}",
                function_name=func,
                **settings,
            )
        )
    print("\n")  # Just to add space between different functions

Function: feed_sync
{'num_docs': 1000, 'max_connections': 16}
{'num_docs': 1000, 'max_connections': 64}
{'num_docs': 10000, 'max_connections': 16}
{'num_docs': 10000, 'max_connections': 64}
{'num_docs': 100000, 'max_connections': 16}
{'num_docs': 100000, 'max_connections': 64}


Function: feed_async
{'num_docs': 1000, 'max_connections': 16}
{'num_docs': 1000, 'max_connections': 64}
{'num_docs': 10000, 'max_connections': 16}
{'num_docs': 10000, 'max_connections': 64}
{'num_docs': 100000, 'max_connections': 16}
{'num_docs': 100000, 'max_connections': 64}


Function: feed_iterable
{'num_docs': 1000, 'max_connections': 64, 'max_workers': 16, 'max_queue_size': 1000}
{'num_docs': 1000, 'max_connections': 64, 'max_workers': 16, 'max_queue_size': 10000}
{'num_docs': 1000, 'max_connections': 64, 'max_workers': 64, 'max_queue_size': 1000}
{'num_docs': 1000, 'max_connections': 64, 'max_workers': 64, 'max_queue_size': 10000}
{'num_docs': 1000, 'max_connections': 128, 'max_workers': 16, 'max_queue_

In [10]:
print(f"Total number of feed_params: {len(feed_params)}")

Total number of feed_params: 36


Now, we will need a way to retrieve the callable function from the function name.


In [11]:
# Get reference to function from string name
def get_func_from_str(func_name: str) -> Callable:
    return globals()[func_name]

### Function to clean up after each feed

For a fair comparison, we will delete the data before feeding it again.


In [12]:
from typing import Iterable, Dict


def delete_data(data: Iterable[Dict]):
    app.feed_iterable(
        iter=data,
        schema="doc",
        namespace="pyvespa-feed",
        operation_type="delete",
        callback=callback,
    )

## Main experiment loop


In [13]:
feed_params[0]

FeedParams(name='1000_16_0_feed_sync', num_docs=1000, max_connections=16, function_name='feed_sync', max_workers=None, max_queue_size=None)

In [14]:
import nest_asyncio

nest_asyncio.apply()

In [15]:
results = []
for params in feed_params:
    print("-" * 50)
    print("Starting feed with params:")
    print(params)
    data = get_dataset(params.num_docs)
    if "async" not in params.function_name:
        feed_result = get_func_from_str(params.function_name)(params=params, data=data)
    else:
        feed_result = asyncio.run(
            get_func_from_str(params.function_name)(params=params, data=data)
        )
    print(feed_result.feed_time)
    results.append(feed_result)
    print("Deleting data")
    delete_data(data)
    print("\n")

--------------------------------------------------
Starting feed with params:
FeedParams(name='1000_16_0_feed_sync', num_docs=1000, max_connections=16, function_name='feed_sync', max_workers=None, max_queue_size=None)


7.820499897003174
Deleting data


--------------------------------------------------
Starting feed with params:
FeedParams(name='1000_64_0_feed_sync', num_docs=1000, max_connections=64, function_name='feed_sync', max_workers=None, max_queue_size=None)
6.688207149505615
Deleting data


--------------------------------------------------
Starting feed with params:
FeedParams(name='10000_16_0_feed_sync', num_docs=10000, max_connections=16, function_name='feed_sync', max_workers=None, max_queue_size=None)
40.09109902381897
Deleting data


--------------------------------------------------
Starting feed with params:
FeedParams(name='10000_64_0_feed_sync', num_docs=10000, max_connections=64, function_name='feed_sync', max_workers=None, max_queue_size=None)
44.30595779418945
Deleting data


--------------------------------------------------
Starting feed with params:
FeedParams(name='100000_16_0_feed_sync', num_docs=100000, max_connections=16, function_name='feed_sync', max_workers=None, max_q

'(ProtocolError('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')), '(Request ID: 160dc6b1-4cdd-41fa-b95c-f9c7c13e97b5)')' thrown while requesting GET https://huggingface.co/datasets/Cohere/wikipedia-2023-11-embed-multilingual-v3/resolve/37feace541fadccf70579e9f289c3cf8e8b186d7/simple/0000.parquet
Retrying in 1s [Retry 1/5].


153.97574615478516
Deleting data


--------------------------------------------------
Starting feed with params:
FeedParams(name='100000_64_0_feed_async', num_docs=100000, max_connections=64, function_name='feed_async', max_workers=None, max_queue_size=None)


'(ProtocolError('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')), '(Request ID: 880c9b07-8a72-4141-9123-13fa5d33db34)')' thrown while requesting GET https://huggingface.co/datasets/Cohere/wikipedia-2023-11-embed-multilingual-v3/resolve/37feace541fadccf70579e9f289c3cf8e8b186d7/simple/0000.parquet
Retrying in 1s [Retry 1/5].


149.27720308303833
Deleting data


--------------------------------------------------
Starting feed with params:
FeedParams(name='1000_64_16_feed_iterable', num_docs=1000, max_connections=64, function_name='feed_iterable', max_workers=16, max_queue_size=1000)
4.185070037841797
Deleting data


--------------------------------------------------
Starting feed with params:
FeedParams(name='1000_64_16_feed_iterable', num_docs=1000, max_connections=64, function_name='feed_iterable', max_workers=16, max_queue_size=10000)
4.1940882205963135
Deleting data


--------------------------------------------------
Starting feed with params:
FeedParams(name='1000_64_64_feed_iterable', num_docs=1000, max_connections=64, function_name='feed_iterable', max_workers=64, max_queue_size=1000)
4.4472739696502686
Deleting data


--------------------------------------------------
Starting feed with params:
FeedParams(name='1000_64_64_feed_iterable', num_docs=1000, max_connections=64, function_name='feed_iterable

'(ProtocolError('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')), '(Request ID: 92188d3d-f950-4d9a-9670-82f6e7a31958)')' thrown while requesting GET https://huggingface.co/datasets/Cohere/wikipedia-2023-11-embed-multilingual-v3/resolve/37feace541fadccf70579e9f289c3cf8e8b186d7/simple/0000.parquet
Retrying in 1s [Retry 1/5].


364.4169690608978
Deleting data


--------------------------------------------------
Starting feed with params:
FeedParams(name='100000_64_64_feed_iterable', num_docs=100000, max_connections=64, function_name='feed_iterable', max_workers=64, max_queue_size=1000)
195.41052627563477
Deleting data


--------------------------------------------------
Starting feed with params:
FeedParams(name='100000_64_64_feed_iterable', num_docs=100000, max_connections=64, function_name='feed_iterable', max_workers=64, max_queue_size=10000)
207.24289894104004
Deleting data


--------------------------------------------------
Starting feed with params:
FeedParams(name='100000_128_16_feed_iterable', num_docs=100000, max_connections=128, function_name='feed_iterable', max_workers=16, max_queue_size=1000)
230.25956678390503
Deleting data


--------------------------------------------------
Starting feed with params:
FeedParams(name='100000_128_16_feed_iterable', num_docs=100000, max_connections=128, function

In [16]:
results

[FeedResult(name='1000_16_0_feed_sync', num_docs=1000, max_connections=16, function_name='feed_sync', max_workers=None, max_queue_size=None, feed_time=7.820499897003174),
 FeedResult(name='1000_64_0_feed_sync', num_docs=1000, max_connections=64, function_name='feed_sync', max_workers=None, max_queue_size=None, feed_time=6.688207149505615),
 FeedResult(name='10000_16_0_feed_sync', num_docs=10000, max_connections=16, function_name='feed_sync', max_workers=None, max_queue_size=None, feed_time=40.09109902381897),
 FeedResult(name='10000_64_0_feed_sync', num_docs=10000, max_connections=64, function_name='feed_sync', max_workers=None, max_queue_size=None, feed_time=44.30595779418945),
 FeedResult(name='100000_16_0_feed_sync', num_docs=100000, max_connections=16, function_name='feed_sync', max_workers=None, max_queue_size=None, feed_time=477.7616879940033),
 FeedResult(name='100000_64_0_feed_sync', num_docs=100000, max_connections=64, function_name='feed_sync', max_workers=None, max_queue_siz

In [17]:
# Create a pandas DataFrame with the results
import pandas as pd

df = pd.DataFrame([result.__dict__ for result in results])
df["requests_per_second"] = df["num_docs"] / df["feed_time"]
df

,name,num_docs,max_connections,function_name,max_workers,max_queue_size,feed_time,requests_per_second
0,1000_16_0_feed_sync,1000,16,feed_sync,NaN,NaN,7.820500,127.869064
1,1000_64_0_feed_sync,1000,64,feed_sync,NaN,NaN,6.688207,149.516900
2,10000_16_0_feed_sync,10000,16,feed_sync,NaN,NaN,40.091099,249.431925
3,10000_64_0_feed_sync,10000,64,feed_sync,NaN,NaN,44.305958,225.703280
4,100000_16_0_feed_sync,100000,16,feed_sync,NaN,NaN,477.761688,209.309374
5,100000_64_0_feed_sync,100000,64,feed_sync,NaN,NaN,569.611136,175.558366
6,1000_16_0_feed_async,1000,16,feed_async,NaN,NaN,3.837450,260.589713
7,1000_64_0_feed_async,1000,64,feed_async,NaN,NaN,3.610098,277.000781
8,10000_16_0_feed_async,10000,16,feed_async,NaN,NaN,15.473620,646.261188
9,10000_64_0_feed_async,10000,64,feed_async,NaN,NaN,14.655987,682.315015


In [18]:
df["function_code"] = df["function_name"].astype("category").cat.codes

In [ ]:
!pip3 install plotly

In [45]:
import plotly.express as px


def plot_performance(df: pd.DataFrame):
    # Create a scatter plot with logarithmic scale for both axes using Plotly Express
    fig = px.scatter(
        df,
        x="num_docs",
        y="requests_per_second",
        color="function_name",  # Defines color based on different functions
        log_x=True,  # Set x-axis to logarithmic scale
        log_y=False,  # If you also want the y-axis in logarithmic scale, set this to True
        title="Performance: Requests per Second vs. Number of Documents",
        labels={  # Customizing axis labels
            "num_docs": "Number of Documents",
            "requests_per_second": "Requests per Second",
            "max_workers": "max_workers",
            "max_queue_size": "max_queue_size",
            "max_connections": "max_connections",
        },
        template="plotly_white",  # This sets the style to a white background, adhering to Tufte's minimalist principles
        hover_data=[
            "max_workers",
            "max_queue_size",
            "max_connections",
        ],  # Additional information to show on hover
    )

    # Update layout for better readability, similar to 'talk' context in Seaborn
    fig.update_layout(
        font=dict(
            size=16,  # Adjusting font size for better visibility, similar to 'talk' context
        ),
        legend_title_text="Function Details",  # Custom legend title
        legend=dict(
            title_font_size=16,
            x=800,  # Adjusting legend position similar to bbox_to_anchor in Matplotlib
            xanchor="auto",
            y=1,
            yanchor="auto",
        ),
        width=800,  # Adjusting width of the plot
    )
    fig.update_xaxes(
        tickvals=[1000, 10000, 100000],  # Set specific tick values
        ticktext=["1k", "10k", "100k"],  # Set corresponding tick labels
    )

    fig.update_traces(
        marker=dict(size=12, opacity=0.7)
    )  # Adjust marker size and opacity
    # Show plot
    fig.show()


plot_performance(df)

Interesting. Let's try to summarize the insights we got from this experiment:

- The `feed_sync` method is the slowest, and does not benefit much from increasing 'max_connections'.
  As there is no concurrency, and each request is blocking, this will be bound by the network latency, which in many cases will be a lot higher than when running against a local VespaDocker instance.
- The `feed_async` method is the fastest, and benefits slightly from increasing 'max_connections' regardless of the number of documents. This method is non-blocking, and will likely be even more beneficial when running against a remote Vespa instance, such as [Vespa Cloud](https://cloud.vespa.ai/), when network latency is higher.
- The `feed_iterable` performance is somewhere in between the other two methods, and benefits a lot from increasing `num_workers` when the number of documents increases.

We have not looked at multiprocessing, but there is definitively room to utilize more cores to improve further upon these results.
But there is one alternative that it is interesting to compare against, the Vespa CLI.


## Feeding with Vespa CLI

[Vespa CLI](https://docs.vespa.ai/en/vespa-cli) is a command-line interface for interacting with Vespa.

Among many useful features are a `vespa feed` command that is the recommended way of feeding large datasets into Vespa.
This is optimized for high feeding performance, and it will be interesting to get a feel for how performant feeding to a local Vespa instance is using the CLI.


## Prepare the data for Vespa CLI

Vespa CLI can feed data from either many .json files or a single .jsonl file with many documents.
The json format needs to be in the following format:

```json
{
  "put": "id:namespace:document-type::document-id",
  "fields": {
    "field1": "value1",
    "field2": "value2"
  }
}
```

Where, `put` is the document operation in this case. Other allowed operations are `get`, `update` and `remove`.

For reference, see https://docs.vespa.ai/en/vespa-cli#cheat-sheet

### Getting the datasets as .jsonl files

Now, let`s save the dataset to 3 different jsonl files of 1k, 10k, and 100k documents.


In [38]:
for n in num_docs:
    print(f"Getting dataset with {n} docs...")
    # First, let's load the dataset in non-streaming mode this time, as we want to save it to a jsonl file
    dataset_cli = load_dataset(
        "Cohere/wikipedia-2023-11-embed-multilingual-v3",
        "simple",
        split=f"train[:{n}]",  # Notice the slicing here, see https://huggingface.co/docs/datasets/loading#slice-splits
        streaming=False,
    )
    # Map to the format expected by the CLI.
    # Note that this differs a little bit from the format expected by the Python API.
    dataset_cli = dataset_cli.map(
        lambda x: {
            "put": f"id:pyvespa-feed:doc::{x['_id']}-json",
            "fields": {"text": x["text"]},
        }
    ).select_columns(["put", "fields"])
    # Save to a jsonl file
    assert len(dataset_cli) == n
    dataset_cli.to_json(f"vespa_feed-{n}.json", orient="records", lines=True)

Getting dataset with 1000 docs...


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Getting dataset with 10000 docs...


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Getting dataset with 100000 docs...


Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

Let's look at the first line of one of the saved files to verify the format.


In [39]:
from pprint import pprint
import json

with open("vespa_feed-1000.json", "r") as f:
    sample = f.readline()
    pprint(json.loads(sample))

{'fields': {'text': 'April (Apr.) is the fourth month of the year in the '
                    'Julian and Gregorian calendars, and comes between March '
                    'and May. It is one of the four months to have 30 days.'},
 'put': 'id:pyvespa-feed:doc::20231101.simple_1_0-json'}


Ok, now we are ready to feed the data using Vespa CLI.
We also want to capture the output of feed statistics for each file.


In [41]:
cli_results = {}
for n in num_docs:
    print(f"Feeding {n} docs...")
    output_list = !vespa feed vespa_feed-{n}.json
    results = json.loads("".join(output_list))
    pprint(results)
    cli_results[n] = results

Feeding 1000 docs...
{'feeder.error.count': 0,
 'feeder.inflight.count': 0,
 'feeder.ok.count': 1000,
 'feeder.ok.rate': 1000.0,
 'feeder.operation.count': 1000,
 'feeder.seconds': 0.246,
 'http.exception.count': 0,
 'http.request.MBps': 0.293,
 'http.request.bytes': 293011,
 'http.request.count': 1000,
 'http.response.MBps': 0.129,
 'http.response.bytes': 129388,
 'http.response.code.counts': {'200': 1000},
 'http.response.count': 1000,
 'http.response.error.count': 0,
 'http.response.latency.millis.avg': 149,
 'http.response.latency.millis.max': 237,
 'http.response.latency.millis.min': 45}
Feeding 10000 docs...
{'feeder.error.count': 0,
 'feeder.inflight.count': 0,
 'feeder.ok.count': 10000,
 'feeder.ok.rate': 10000.0,
 'feeder.operation.count': 10000,
 'feeder.seconds': 0.911,
 'http.exception.count': 0,
 'http.request.MBps': 2.906,
 'http.request.bytes': 2905878,
 'http.request.count': 10000,
 'http.response.MBps': 1.317,
 'http.response.bytes': 1317226,
 'http.response.code.count

In [42]:
cli_results

{1000: {'feeder.operation.count': 1000,
  'feeder.seconds': 0.246,
  'feeder.ok.count': 1000,
  'feeder.ok.rate': 1000.0,
  'feeder.error.count': 0,
  'feeder.inflight.count': 0,
  'http.request.count': 1000,
  'http.request.bytes': 293011,
  'http.request.MBps': 0.293,
  'http.exception.count': 0,
  'http.response.count': 1000,
  'http.response.bytes': 129388,
  'http.response.MBps': 0.129,
  'http.response.error.count': 0,
  'http.response.latency.millis.min': 45,
  'http.response.latency.millis.avg': 149,
  'http.response.latency.millis.max': 237,
  'http.response.code.counts': {'200': 1000}},
 10000: {'feeder.operation.count': 10000,
  'feeder.seconds': 0.911,
  'feeder.ok.count': 10000,
  'feeder.ok.rate': 10000.0,
  'feeder.error.count': 0,
  'feeder.inflight.count': 0,
  'http.request.count': 10000,
  'http.request.bytes': 2905878,
  'http.request.MBps': 2.906,
  'http.exception.count': 0,
  'http.response.count': 10000,
  'http.response.bytes': 1317226,
  'http.response.MBps': 

In [44]:
# Let's add the CLI results to the DataFrame
df_cli = pd.DataFrame(
    [
        {
            "name": f"{n}_cli",
            "num_docs": n,
            "max_connections": "unknown",
            "function_name": "cli",
            "max_workers": "unknown",
            "max_queue_size": "n/a",
            "feed_time": result["feeder.seconds"],
        }
        for n, result in cli_results.items()
    ]
)
df_cli["requests_per_second"] = df_cli["num_docs"] / df_cli["feed_time"]
df_cli

,name,num_docs,max_connections,function_name,max_workers,max_queue_size,feed_time,requests_per_second
0,1000_cli,1000,unknown,cli,unknown,n/a,0.246,4065.040650
1,10000_cli,10000,unknown,cli,unknown,n/a,0.911,10976.948408
2,100000_cli,100000,unknown,cli,unknown,n/a,5.959,16781.339151


In [48]:
df_total = pd.concat([df, df_cli])

plot_performance(df_total)

As you can tell, the CLI is orders of magnitude faster.


## Conclusion


- Prefer to use the CLI if you care about performance. 🚀
- If you are using Python, prefer the async method, as it is the fastest way to feed data using `pyvespa`.


## Cleanup


In [ ]:
vespa_docker.container.stop()
vespa_docker.container.remove()

## Next steps

Check out some of the other
[examples](https://pyvespa.readthedocs.io/en/latest/examples.html) in the documentation.
